# 🗺️ SAT POI Viewer with WD + OSM
This notebook loads POIs (e.g. toilets, water points) from Wikidata with optional OSM matches and renders them on a Folium map with filtering options.

In [7]:
import time
from datetime import datetime

start_time = time.time()
print("Start:", datetime.now().strftime("%Y-%m-%d %H:%M:%S"))
    

Start: 2025-07-24 19:14:44


In [8]:
import pandas as pd

In [9]:
# Run a live SPARQL query to get SAT POIs with optional OSM links
from SPARQLWrapper import SPARQLWrapper, JSON

endpoint_url = "https://query.wikidata.org/sparql"
query = """
SELECT ?item ?itemLabel ?coord ?OSMnode ?OSMway ?OSMrel WHERE {
  ?item wdt:P6104 wd:Q134294510; wdt:P625 ?coord.
  OPTIONAL { ?item wdt:P10689 ?OSMway. }
  OPTIONAL { ?item wdt:P402 ?OSMrel. }
  OPTIONAL { ?item wdt:P11693 ?OSMnode. }
  SERVICE wikibase:label { bd:serviceParam wikibase:language 'sv,en'. }
}
"""

sparql = SPARQLWrapper(endpoint_url)
sparql.setQuery(query)
sparql.setReturnFormat(JSON)
results = sparql.query().convert()

import re

data = []
for res in results["results"]["bindings"]:
    label = res["itemLabel"]["value"]
    coord = res["coord"]["value"]  # e.g. Point(18.858 59.756)
    lon, lat = map(float, re.findall(r"[-+]?\d+\.\d+", coord))
    node = res.get("OSMnode", {}).get("value")
    way = res.get("OSMway", {}).get("value")
    rel = res.get("OSMrel", {}).get("value")

    if node:
        source = "OSM"
        osm_url = f"https://www.openstreetmap.org/node/{node}"
    elif way:
        source = "OSM"
        osm_url = f"https://www.openstreetmap.org/way/{way}"
    elif rel:
        source = "OSM"
        osm_url = f"https://www.openstreetmap.org/relation/{rel}"
    else:
        source = "WD"
        osm_url = None

    if node or way or rel:
        source = "both"

    data.append({"label": label, "lat": lat, "lon": lon, "source": source, "osm_url": osm_url})

df = pd.DataFrame(data)
df.head()

,label,lat,lon,source,osm_url
0,Knicksand grillplats,58.917789,18.165038,both,https://www.openstreetmap.org/node/12722761765
1,Knicksand grillplats,58.917563,18.165444,both,https://www.openstreetmap.org/node/12722789034
2,"Sandön, grillplats",59.278198,18.909647,both,https://www.openstreetmap.org/node/12752418489
3,"Myrudden, grillplats",59.664687,18.911037,both,https://www.openstreetmap.org/node/12873021137
4,Lidö brygga,59.770089,19.079149,both,https://www.openstreetmap.org/node/1823005981


In [18]:
df

,label,lat,lon,source,osm_url
0,Toilet WD,59.750,18.850,WD,None
1,Toilet OSM,59.752,18.855,OSM,https://www.openstreetmap.org/node/123
2,Water both,59.760,18.820,both,https://www.openstreetmap.org/node/456


In [10]:
import folium
import pandas as pd
from folium import FeatureGroup
from shapely.geometry import Point
from IPython.display import display

# Sample POIs (normally loaded via SPARQL)
data = [
    {"label": "Toilet WD", "lat": 59.75, "lon": 18.85, "source": "WD", "osm_url": None},
    {"label": "Toilet OSM", "lat": 59.752, "lon": 18.855, "source": "OSM", "osm_url": "https://www.openstreetmap.org/node/123"},
    {"label": "Water both", "lat": 59.76, "lon": 18.82, "source": "both", "osm_url": "https://www.openstreetmap.org/node/456"},
]
df = pd.DataFrame(data)

In [11]:
# Create map
m = folium.Map(location=[59.75, 18.85], zoom_start=13)

# Add groups
fg_wd = FeatureGroup(name="Wikidata")
fg_osm = FeatureGroup(name="OSM")
fg_both = FeatureGroup(name="WD + OSM")

# Add markers to appropriate group
for _, row in df.iterrows():
    marker = folium.Marker(
        [row["lat"], row["lon"]],
        popup=f"<b>{row['label']}</b><br>{row['osm_url'] or ''}",
        icon=folium.Icon(color="blue" if row["source"] == "WD" else "green" if row["source"] == "OSM" else "purple")
    )
    if row["source"] == "WD":
        marker.add_to(fg_wd)
    elif row["source"] == "OSM":
        marker.add_to(fg_osm)
    else:
        marker.add_to(fg_both)

# Add groups to map
fg_wd.add_to(m)
fg_osm.add_to(m)
fg_both.add_to(m)
folium.LayerControl().add_to(m)
m

In [12]:
# End timer and calculate duration
end_time = time.time()
elapsed_time = end_time - start_time

# Print current date and total time
print("Date:", datetime.now().strftime("%Y-%m-%d %H:%M:%S"))
print("Total time elapsed: {:.2f} seconds".format(elapsed_time))

Date: 2025-07-24 19:14:52
Total time elapsed: 7.46 seconds
